### Carga y validación inicial del fichero de coordenadas (mallado)

In [1]:
import pandas as pd
from datetime import datetime
import os
import re
import time
import requests
import pandas as pd
from urllib.parse import urlencode
import io

fecha = datetime.now().strftime('%Y%m%d_%H%M')


df_coords = pd.read_csv("mallado.csv")
print(df_coords.shape)      # filas y columnas
print(df_coords.columns)    # nombres de columnas
df_coords.head()            # primeras filas


(27, 3)
Index(['poblacion_total', 'lat', 'lon'], dtype='object')


,poblacion_total,lat,lon
0,14458,43.010088,-9.188413
1,260901,37.257363,-7.200928
2,217987,37.965370,-7.138822
3,20120,39.499208,-7.171370
4,169450,42.033670,-7.933326


### Descarga y consolidación de datos TMY por coordenadas

Descarga los datos TMY de PVGIS para cada punto del mallado, los unifica en un único conjunto de datos y los exporta a un fichero CSV para su análisis posterior.

In [ ]:
tmy_global = []

for idx, row in df_coords.iterrows():
    lat = row["lat"]
    lon = row["lon"]

    print(f"Descargando TMY para lat={lat}, lon={lon} ...")

    url = "https://re.jrc.ec.europa.eu/api/tmy"
    params = {
        "lat": lat,
        "lon": lon,
        "outputformat": "csv"
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        lines = response.text.splitlines()

        # buscar la línea donde empieza la tabla horaria
        start_idx = None
        for i, line in enumerate(lines):
            if line.startswith("time(UTC)"):
                start_idx = i
                break

        if start_idx is None:
            print(f"  !! No se encontró cabecera 'time(UTC)' para {lat}, {lon}")
            continue

        csv_data = "\n".join(lines[start_idx:])
        df_tmy = pd.read_csv(io.StringIO(csv_data))

        # añadir coordenadas
        df_tmy["lat"] = lat
        df_tmy["lon"] = lon

        tmy_global.append(df_tmy)
        print(f"  OK, filas: {df_tmy.shape[0]}")
    else:
        print(f"  !! Error HTTP {response.status_code} para {lat}, {lon}")

    time.sleep(1)  # pequeña pausa para no abusar de la API

# unimos todo y exportamos
if tmy_global:
    df_final = pd.concat(tmy_global, ignore_index=True)
    df_final.to_csv("TMY_all_coordinates.csv", index=False)
    print("Archivo creado: TMY_all_coordinates.csv")
    print("Filas totales:", df_final.shape[0])
else:
    print("No se ha descargado ningún TMY.")

Descargando TMY para lat=43.01008776, lon=-9.188413044333332 ...
  OK, filas: 8770
Descargando TMY para lat=37.257363038571434, lon=-7.200928419999999 ...
  OK, filas: 8770
Descargando TMY para lat=37.965370157307696, lon=-7.138822035 ...
  OK, filas: 8770
Descargando TMY para lat=39.49920750818182, lon=-7.171370430363637 ...
  OK, filas: 8770
Descargando TMY para lat=42.033670423414634, lon=-7.933325822585366 ...
  OK, filas: 8770
Descargando TMY para lat=42.82670514307985, lon=-7.964607534338403 ...
  OK, filas: 8770
Descargando TMY para lat=36.841508026165414, lon=-5.42390867006015 ...
  OK, filas: 8770
Descargando TMY para lat=38.095268199430606, lon=-5.974638156576512 ...
  OK, filas: 8770
Descargando TMY para lat=39.98708255200837, lon=-5.740223103259415 ...
  OK, filas: 8770
Descargando TMY para lat=41.29388043232274, lon=-5.572345771344744 ...
  OK, filas: 8770
Descargando TMY para lat=42.69003347615384, lon=-5.659510861418462 ...
  OK, filas: 8770
Descargando TMY para lat=37.0

### Limpieza, transformación y agregación diaria de datos TMY por coordenada

Este bloque carga los datos TMY consolidados, limpia y convierte las fechas a formato `datetime`, transforma las variables climáticas a tipo numérico y realiza una agregación diaria por coordenada (lat, lon), obteniendo valores mínimos, máximos y medios. El resultado se exporta en un fichero diario agregado listo para su análisis posterior.

In [3]:
df_tmy_all = pd.read_csv("TMY_all_coordinates.csv")

print(df_tmy_all.shape)      # filas, columnas
print(df_tmy_all.head())     # primeras filas
print(df_tmy_all["lat"].nunique(), df_tmy_all["lon"].nunique())  # nº de puntos

df_tmy_all["date_utc"] = pd.to_datetime(
    df_tmy_all["time(UTC)"],
    format="%Y%m%d:%H%M",
    errors="coerce"   # convierte lo que no encaja en NaT
)

df_tmy_clean = df_tmy_all.dropna(subset=["date_utc"]).copy()

print(df_tmy_all.shape, "-> antes")
print(df_tmy_clean.shape, "-> después de limpiar")


# Agregado diario por coordenada
cols_to_num = ["T2m", "G(h)", "WS10m"]

for c in cols_to_num:
    df_tmy_clean[c] = pd.to_numeric(df_tmy_clean[c], errors="coerce")


df_daily = (
    df_tmy_clean
    .groupby(["lat", "lon", df_tmy_clean["date_utc"].dt.date])
    .agg(
        Temp_min_C=("T2m", "min"),
        Temp_max_C=("T2m", "max"),
        Temp_media_C=("T2m", "mean"),
        RadiacionGlobal_media_Wm2=("G(h)", "mean"),
        Viento_media_ms=("WS10m", "mean")
    )
    .reset_index()
    .rename(columns={"date_utc": "date_utc"})
)

print(df_daily.shape)
df_daily.head()


df_daily.to_csv("TMY_daily_agg.csv", index=False)
print("Archivo guardado: TMY_daily_agg.csv")


(236790, 12)
       time(UTC)   T2m     RH  G(h)  Gb(n)  Gd(h)   IR(h)  WS10m  WD10m  \
0  20210101:0000   8.8  84.77   0.0   -0.0    0.0  314.66   5.91  352.0   
1  20210101:0100  8.22  85.33   0.0   -0.0    0.0  308.08   5.64  339.0   
2  20210101:0200  7.64  85.89   0.0   -0.0    0.0  301.51   5.37  329.0   
3  20210101:0300  7.06  86.45   0.0   -0.0    0.0  294.93   5.10  318.0   
4  20210101:0400  6.48  87.01   0.0   -0.0    0.0  288.36   4.82  322.0   

        SP        lat       lon  
0  98980.0  43.010088 -9.188413  
1  98910.0  43.010088 -9.188413  
2  98890.0  43.010088 -9.188413  
3  98840.0  43.010088 -9.188413  
4  98820.0  43.010088 -9.188413  
27 27
(236790, 13) -> antes
(236520, 13) -> después de limpiar
(9855, 8)
Archivo guardado: TMY_daily_agg.csv
